In [1]:
%matplotlib inline

import numpy
import pandas
import geopandas
import libpysal as ps
import seaborn
import contextily
import pointpats
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN
from pointpats import centrography
from matplotlib.patches import Ellipse
from matplotlib.patches import Polygon, Circle, Rectangle
from descartes import PolygonPatch


db = pandas.read_csv('tramas_export_1645791400550.csv')
centros_db = pandas.read_csv('centros.csv')
centros_db.set_index('grupo')
relaciones_db = pandas.read_csv('relacion.csv')
db['lng'] = db['longitude'].where(db['ns'] == 'S', -1*db['longitude'])
db['lat'] = db['latitude'].where(db['ew'] == 'W', -1*db['latitude'])

fulldb = db.set_index('deviceId').join(relaciones_db.set_index('id'))

has_location =  fulldb['eventLength']>77
loc_db = fulldb[has_location]


for grupo in range(1,4):
    for modulo in ['LC76F','L76LB']:
        has_grupo = loc_db['grupo']==grupo
        grupo_db = loc_db[has_grupo]
        has_modulo = grupo_db['modulo']==modulo
        grupo_modulo_db = grupo_db[has_modulo]
        outfile = 'Grupo{0}_{1}'.format(grupo,modulo)
        print(outfile)
        # Generate scatter plot
        # seaborn.jointplot(x='lng', y='lat', data=loc_db, s=0.5);

        # Generate scatter plot
        joint_axes = seaborn.jointplot(
            x='lng', y='lat', data=grupo_modulo_db, s=1, height=15
        )
        contextily.add_basemap(
            joint_axes.ax_joint,
            crs="EPSG:4326",
            source=contextily.providers.CartoDB.PositronNoLabels
        )

        mean_center = centrography.mean_center(grupo_modulo_db[['lng', 'lat']])
        med_center = centrography.euclidean_median(grupo_modulo_db[['lng', 'lat']])

        # Generate scatter plot
        joint_axes = seaborn.jointplot(
            x='lng', y='lat', data=grupo_modulo_db, s=1, height=10
        )
        # Add mean point and marginal lines
        joint_axes.ax_joint.scatter(
            *mean_center, color='red', marker='x', s=50, label='Mean Center'
        )
        joint_axes.ax_marg_x.axvline(mean_center[0], color='red')
        joint_axes.ax_marg_y.axhline(mean_center[1], color='red')
        # Add median point and marginal lines
        joint_axes.ax_joint.scatter(
            *med_center, color='limegreen', marker='o', s=50, label='Median Center'
        )
        joint_axes.ax_marg_x.axvline(med_center[0], color='limegreen')
        joint_axes.ax_marg_y.axhline(med_center[1], color='limegreen')
        # Legend
        joint_axes.ax_joint.legend()
        # Add basemap
        contextily.add_basemap(
            joint_axes.ax_joint,
            crs="EPSG:4326",
            source=contextily.providers.CartoDB.PositronNoLabels
        )
        # Clean axes
        joint_axes.ax_joint.set_axis_off()
        # Display
        plt.show()
        coordinates = grupo_modulo_db[['lng','lat']].values
        # print(coordinates)

        convex_hull_vertices = centrography.hull(coordinates)
        #(radius, center), inset, removed, constraints = pointpats.skyum(coordinates)
        #(center_x, center_y), radius = centrography.minimum_bounding_circle(coordinates)

        major, minor, rotation = centrography.ellipse(grupo_modulo_db[['lng','lat']])
        # Set up figure and axis
        f, ax = plt.subplots(1, figsize=(15, 15))
        # Plot photograph points
        ax.scatter(grupo_modulo_db['lng'], grupo_modulo_db['lat'], s=1)
        ax.scatter(*mean_center, color='red', marker='x', label='Mean Center')
        ax.scatter(*med_center, color='limegreen', marker='o', label='Median Center')

        # Construct the standard ellipse using matplotlib
        ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                        width=major*2, # centrography.ellipse only gives half the axis
                        height=minor*2, 
                        angle = 180-numpy.rad2deg(rotation), # Angles for this are in degrees, not radians
                        facecolor='none', 
                        edgecolor='red', linestyle='--',
                        label='Std. Ellipse')
        ax.add_patch(ellipse)

        # Construct the standard ellipse using matplotlib
        ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                        width=3*major*2, # centrography.ellipse only gives half the axis
                        height=3*minor*2, 
                        angle = 180-numpy.rad2deg(rotation), # Angles for this are in degrees, not radians
                        facecolor='none', 
                        edgecolor='green', linestyle='--',
                        label='3 Std. Ellipse')
        ax.add_patch(ellipse)

        # add a blue convex hull
        ax.add_patch(
            plt.Polygon(
                convex_hull_vertices, 
                closed=True, 
                edgecolor='blue', 
                facecolor='none', 
                linestyle=':', 
                linewidth=2,
                label='Convex Hull'
            )
        )
        # and a red minimum bounding circle
        # circ_patch = Circle(
        #     (center_x, center_y), 
        #     radius=radius,
        #     edgecolor='red', 
        #     facecolor='none', 
        #     linewidth=2,
        #     label='Min Bounding Circle'
        # )
        ax.legend()
        # Display
        # Add basemap
        contextily.add_basemap(
            ax,
            crs="EPSG:4326",
            source= contextily.providers.OpenStreetMap.Mapnik
        )
        plt.show()

        qstat = pointpats.QStatistic(coordinates)
        qstat.plot()

ModuleNotFoundError: No module named 'geopandas'